In [1]:
from scipy.stats import entropy
import warnings
warnings.filterwarnings("ignore")
import yfinance as yf
import h5py
import datetime
import requests
from io import StringIO
from IPython.core.display import HTML
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
import pandas as pd
from pyquery import PyQuery as pq
import numpy as np
import bs4
from bs4 import BeautifulSoup
import time
import math
import random
import statistics
import schedule
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import requests
from io import StringIO
from bs4 import BeautifulSoup
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn import preprocessing
import xgboost as xgb
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import talib
from talib import abstract,MACD,RSI,SMA,DEMA,EMA,HT_TRENDLINE,ADX,APO,NATR,HT_DCPERIOD,CDL2CROWS
from tqdm import tqdm 

def Create_DeTrend_DeSTD_Data(sourse_location,save_location,window):
    資料更新24(sourse_location)

    O_data=pd.read_csv(sourse_location)
    O_data["STOCK_ID"]=O_data["STOCK_ID"].astype(str)
    O_data=O_data[["STOCK_ID","Date","Open","High","Low","Close"]]
    SLIST=O_data["STOCK_ID"].value_counts().index
    CLIST=["Open","High","Low","Close"]
    Detrend2=Detrend(O_data,window,SLIST,CLIST).dropna()
    O_data=Detrend2[['Open_R_Trend','High_R_Trend','Low_R_Trend','Close_R_Trend','Open_R_STD','High_R_STD','Low_R_STD','Close_R_STD','STOCK_ID', 'Date', 'Open', 'High', 'Low', 'Close']]
    R_data=Detrend2[['Open_R_Trend','High_R_Trend','Low_R_Trend','Close_R_Trend','Open_R_STD','High_R_STD','Low_R_STD','Close_R_STD','STOCK_ID', 'Date', 'Open_R', 'High_R', 'Low_R', 'Close_R']].rename({'Open_R':'Open', 
                                                                                        'High_R':'High', 
                                                                                        'Low_R':'Low', 
                                                                                        'Close_R':'Close'}, axis=1)
    D1_data=Detrend2[['Open_R_Trend','High_R_Trend','Low_R_Trend','Close_R_Trend','Open_R_STD','High_R_STD','Low_R_STD','Close_R_STD','STOCK_ID', 'Date', 'Detrend_Open', 'Detrend_High', 'Detrend_Low', 'Detrend_Close']].rename({'Detrend_Open':'Open', 
                                                                                        'Detrend_High':'High', 
                                                                                        'Detrend_Low':'Low', 
                                                                                        'Detrend_Close':'Close'}, axis=1)
    D2_data=Detrend2[['Open_R_Trend','High_R_Trend','Low_R_Trend','Close_R_Trend','Open_R_STD','High_R_STD','Low_R_STD','Close_R_STD','STOCK_ID', 'Date', 'DeSTD_Open', 'DeSTD_High', 'DeSTD_Low', 'DeSTD_Close']].rename({'DeSTD_Open':'Open', 
                                                                                        'DeSTD_High':'High', 
                                                                                        'DeSTD_Low':'Low', 
                                                                                       'DeSTD_Close':'Close'}, axis=1)
    TAFEATURE(O_data,SLIST).to_csv("O_data.csv",index=None)
    TAFEATURE(R_data,SLIST).to_csv("R_data.csv",index=None)
    TAFEATURE(D1_data,SLIST).to_csv("D1_data.csv",index=None)
    TAFEATURE(D2_data,SLIST).to_csv("D2_data.csv",index=None)
    
def TAFEATURE(data, SLIST):
    temp=pd.DataFrame()
    with tqdm(total=len(SLIST)) as pbar:
        for i in SLIST:
            temp2=data[data.STOCK_ID==i].sort_values("Date")
            close=temp2["Close"]
            high=temp2["High"]
            low=temp2["Low"]
            Open=temp2["Open"]
            temp2["MACD"] = MACD(close, fastperiod=5, slowperiod=10,signalperiod=3)[0]
            temp2["RSI"]=RSI(close, timeperiod=5)      
            temp2["ADX"]=ADX(high, low, close, timeperiod=14)
            temp2["APO"]=APO(close, fastperiod=5, slowperiod=10, matype=0)
            temp2["NATR"]=NATR(high, low, close, timeperiod=10)
            temp2["CDL2CROWS"]=talib.CDL2CROWS(Open, high, low, close)
            temp2["CDL3BLACKCROWS"]=talib.CDL3BLACKCROWS(Open, high, low, close)
            temp2["CDL3INSIDE"]=talib.CDL3INSIDE(Open, high, low, close)
            temp2['CDL3STARSINSOUTH'] =talib.CDL3STARSINSOUTH(Open, high, low, close)
            temp2['CDL3WHITESOLDIERS'] =talib.CDL3WHITESOLDIERS(Open, high, low, close)
            temp2['CDLABANDONEDBABY'] =talib.CDLABANDONEDBABY(Open, high, low, close, penetration=0)
            temp2['CDLADVANCEBLOCK'] =talib.CDLADVANCEBLOCK(Open, high, low, close)
            temp2['CDLBELTHOLD'] =talib.CDLBELTHOLD(Open, high, low, close)
            temp2['CDLBREAKAWAY'] =talib.CDLBREAKAWAY(Open, high, low, close)
            temp2['CDLCLOSINGMARUBOZU'] =talib.CDLCLOSINGMARUBOZU(Open, high, low, close)
            temp2['CDLCONCEALBABYSWALL'] =talib.CDLCONCEALBABYSWALL(Open, high, low, close)
            temp2['CDLCOUNTERATTACK'] =talib.CDLCOUNTERATTACK(Open, high, low, close)
            temp2['CDLDARKCLOUDCOVER'] =talib.CDLDARKCLOUDCOVER(Open, high, low, close, penetration=0)
            temp2['CDLDOJI'] =talib.CDLDOJI(Open, high, low, close)
            temp2['CDLDOJISTAR'] =talib.CDLDOJISTAR(Open, high, low, close)
            temp2['CDLDRAGONFLYDOJI'] =talib.CDLDRAGONFLYDOJI(Open, high, low, close)
            temp2['CDLENGULFING'] =talib.CDLENGULFING(Open, high, low, close)
            temp2['CDLEVENINGDOJISTAR'] =talib.CDLEVENINGDOJISTAR(Open, high, low, close, penetration=0)
            temp2['CDLEVENINGSTAR'] =talib.CDLEVENINGSTAR(Open, high, low, close, penetration=0)
            temp2['CDLGAPSIDESIDEWHITE'] =talib.CDLGAPSIDESIDEWHITE(Open, high, low, close)
            temp2['CDLGRAVESTONEDOJI'] =talib.CDLGRAVESTONEDOJI(Open, high, low, close)
            temp2['CDLHAMMER'] =talib.CDLHAMMER(Open, high, low, close)
            temp2['CDLHANGINGMAN'] =talib.CDLHANGINGMAN(Open, high, low, close)
            temp2['CDLHARAMI'] =talib.CDLHARAMI(Open, high, low, close)
            temp2['CDLHARAMICROSS'] =talib.CDLHARAMICROSS(Open, high, low, close)
            temp2['CDLHIGHWAVE'] =talib.CDLHIGHWAVE(Open, high, low, close)
            temp2['CDLHIKKAKE'] =talib.CDLHIKKAKE(Open, high, low, close)
            temp2['CDLHIKKAKEMOD'] =talib.CDLHIKKAKEMOD(Open, high, low, close)
            temp2['CDLHOMINGPIGEON'] =talib.CDLHOMINGPIGEON(Open, high, low, close)
            temp2['CDLIDENTICAL3CROWS'] =talib.CDLIDENTICAL3CROWS(Open, high, low, close)
            temp2['CDLINNECK'] =talib.CDLINNECK(Open, high, low, close)
            temp2['CDLINVERTEDHAMMER'] =talib.CDLINVERTEDHAMMER(Open, high, low, close)
            temp2['CDLKICKING'] =talib.CDLKICKING(Open, high, low, close)
            temp2['CDLKICKINGBYLENGTH'] =talib.CDLKICKINGBYLENGTH(Open, high, low, close)
            temp2['CDLLADDERBOTTOM'] =talib.CDLLADDERBOTTOM(Open, high, low, close)
            temp2['CDLLONGLEGGEDDOJI'] =talib.CDLLONGLEGGEDDOJI(Open, high, low, close)
            temp2['CDLLONGLINE'] =talib.CDLLONGLINE(Open, high, low, close)
            temp2['CDLMARUBOZU'] =talib.CDLMARUBOZU(Open, high, low, close)
            temp2['CDLMATCHINGLOW'] =talib.CDLMATCHINGLOW(Open, high, low, close)
            temp2['CDLMATHOLD'] =talib.CDLMATHOLD(Open, high, low, close, penetration=0)
            temp2['CDLMORNINGDOJISTAR'] =talib.CDLMORNINGDOJISTAR(Open, high, low, close, penetration=0)
            temp2['CDLMORNINGSTAR'] =talib.CDLMORNINGSTAR(Open, high, low, close, penetration=0)
            temp2['CDLONNECK'] =talib.CDLONNECK(Open, high, low, close)
            temp2['CDLPIERCING'] =talib.CDLPIERCING(Open, high, low, close)
            temp2['CDLRICKSHAWMAN'] =talib.CDLRICKSHAWMAN(Open, high, low, close)
            temp2['CDLRISEFALL3METHODS'] =talib.CDLRISEFALL3METHODS(Open, high, low, close)
            temp2['CDLSEPARATINGLINES'] =talib.CDLSEPARATINGLINES(Open, high, low, close)
            temp2['CDLSHOOTINGSTAR'] =talib.CDLSHOOTINGSTAR(Open, high, low, close)
            temp2['CDLSHORTLINE'] =talib.CDLSHORTLINE(Open, high, low, close)
            temp2['CDLSPINNINGTOP'] =talib.CDLSPINNINGTOP(Open, high, low, close)
            temp2['CDLSTALLEDPATTERN'] =talib.CDLSTALLEDPATTERN(Open, high, low, close)
            temp2['CDLSTICKSANDWICH'] =talib.CDLSTICKSANDWICH(Open, high, low, close)
            temp2['CDLTAKURI'] =talib.CDLTAKURI(Open, high, low, close)
            temp2['CDLTASUKIGAP'] =talib.CDLTASUKIGAP(Open, high, low, close)
            temp2['CDLTHRUSTING'] =talib.CDLTHRUSTING(Open, high, low, close)
            temp2['CDLTRISTAR'] =talib.CDLTRISTAR(Open, high, low, close)
            temp2['CDLUNIQUE3RIVER'] =talib.CDLUNIQUE3RIVER(Open, high, low, close)
            temp2['CDLUPSIDEGAP2CROWS'] =talib.CDLUPSIDEGAP2CROWS(Open, high, low, close)
            temp2['CDLXSIDEGAP3METHODS'] =talib.CDLXSIDEGAP3METHODS(Open, high, low, close)
            pbar.update(1)
            temp=temp.append(temp2)
        
    return temp

def Detrend(O_data,window, SLIST,CLIST):
    ALL=pd.DataFrame()
    for S in SLIST:
        temp=O_data[O_data["STOCK_ID"]==S]
        for C in CLIST:
            temp[C+"_R"]=(temp[C]-temp[C].shift(1))/temp[C]
            temp[C+"_R_Trend"]=temp[C+"_R"].rolling(window).mean()
            temp["Detrend_"+C]=temp[C+"_R"]-temp[C+"_R_Trend"]
            temp[C+"_R_STD"]=temp[C+"_R"].rolling(window).std()
            temp["DeSTD_"+C]=temp["Detrend_"+C]/temp[C+"_R_STD"]
        ALL=ALL.append(temp)
    return ALL

def 資料更新24(location):
    SLIST=[2330, 2454, 2317, 2303, 2881, 2308, 1301, 2882, 1303, 2891, 2412,
       2002, 2886, 3711, 2884, '202112TX', 5871, 1216, 2603, 6415, 2885,
       3034, 1326, 2379]
    historical_data = pd.DataFrame()
    data2=pd.read_csv(location)
    startD=data2["Date"].max()
    startD=str(startD)[0:4]+"-"+str(startD)[4:6]+"-"+str(startD)[6:]
    endD=datetime.datetime.today().date().strftime("%Y-%m-%d")
    for i in SLIST:    

        # 抓取股票資料
        stock_id = str(i) + '.TW'
        data = yf.Ticker(stock_id)
        df = data.history(start=startD,end=endD)

        # 增加股票代號
        df['STOCK_ID'] = str(i)

        # 合併
        historical_data = pd.concat([historical_data, df])
        time.sleep(0.8)
    historical_data["Date"]=historical_data.index.astype(str).str.replace("-","")
    historical_data=historical_data.reset_index(drop=True)
        
    temp=data2.append(historical_data)
    temp["STOCK_ID"]=temp["STOCK_ID"].astype(str)
    temp["Date"]=temp["Date"].astype(int)
    temp=temp.drop_duplicates(["STOCK_ID","Date"])
    temp.to_csv(location,index=None)

def find_index(Stock_list, extract, label,  null) :
    institutional = pd.DataFrame()
    for i in range (len(Stock_list)):
        a = extract[extract[label] == str(Stock_list[i])].index
        if len(a)== 1  :
            institutional=institutional.append(extract.iloc[a])
        elif len(a) == 0 :
            institutional=institutional.append(null)
    return institutional.reset_index(drop=True)

def devide(list1 , list2) :
    out_come = list()
    for i in range (len(list1)):
        if type (list1[i]) != float and  int(list2[i].replace(",", "")) >0 :
            x = format(int(list1[i].replace(",", "")) / int(list2[i].replace(",", "")) ,'.3f')
        else :
            x = 0
        out_come.append(x)
    return out_come

def extract(Stock_list, whole_df , label):
    start = 0
    end = 0
    point = Stock_list[0]
    for i in range (len(whole_df)):
        if whole_df.iloc[i][label] >= str(point):
            #print (whole_df.iloc[i][label] , i)
            if start != 0 :
                end = i
                break
            point = Stock_list[len(Stock_list)-1]
            start = i
    test =  pd.DataFrame(whole_df[start:end+1])
    return test

def check_symbol (stock_df ,label) :
    test_dr = pd.DataFrame()
    for i in range (len(stock_df)):
        if len(stock_df.iloc[i][label]) == 4:
            test_dr = test_dr.append(stock_df[i:i+1]).reset_index(drop=True)
    return test_dr

def SendMail(string,r,title):
    content = MIMEMultipart()  #建立MIMEMultipart物件
    content["subject"] = title  #郵件標題
    content["from"] = "yezifa2005@gmail.com"  #寄件者
    content["to"] = r #收件者
    content.attach(MIMEText(string))  #郵件內容

    with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:  # 設定SMTP伺服器
        try:
            smtp.ehlo()  # 驗證SMTP伺服器
            smtp.starttls()  # 建立加密傳輸
            smtp.login("yezifa2005@gmail.com", "gcvquwtmgdkkiefl")  # 登入寄件者gmail
            smtp.send_message(content)  # 寄送郵件
            print("Complete!")
        except Exception as e:
            print("Error message: ", e)
            
def rise_estimate(df, day , upper , lower):
    i=0
    label = "p" + str(day) + "_" + str(upper)
    list_ = []
    K = 0
    while i < len(df):
        try :
            UP = df['收盤價'][i+1:i+day+1].tolist()
            LP = df['最低價'][i+1:i+day+1].tolist()
            U =(max(UP)-float(df['開盤價'][i+1]))/float(df['開盤價'][i+1]+0.0001)
            L =(min(LP)-float(df['開盤價'][i+1]))/float(df['開盤價'][i+1]+0.0001)
            
            if U > upper :
                if ((L < lower) and ( LP.index(min(LP)) < UP.index(max(UP)))) :
                    list_.append(0)
                else:
                    list_.append(1)
            else :
                list_.append(0)
        except Exception as e:
            print(e, i )
            list_.append(0)
        i+=1
    df[label] = list_
def drop_estimate(df, day , upper , lower):
    i=0
    label = "d" + str(day) + "_" + str(upper)
    list_ = []
    while i < len(df):
        try :
            UP = df['收盤價'][i+1:i+day+1].tolist()
            LP = df['最高價'][i+1:i+day+1].tolist()
            U =(min(UP)-float(df['開盤價'][i+1]))/float(df['開盤價'][i+1]+0.0001)
            L =(max(LP)-float(df['開盤價'][i+1]))/float(df['開盤價'][i+1]+0.0001) 
            
            if ( U < -upper ) :
                if (L > -lower ) and (LP.index(max(LP)) < UP.index(min(UP))):
                    list_.append(0)
                else:
                    list_.append(1)                   
            else :
                list_.append(0)
        except Exception as e:
            print(e, i )
            list_.append(0)
        i+=1
    df[label] = list_
def shift_rise_estimate(df, day , upper ):
    i=0
    label = "pre" + str(day) + "r" + str(upper)
    list_ = [0 for x in range(day)]
    while i < (len(df) -day):
        try :
        
            U =(max(df['最高價'][i+1:i+day+1])-float(df['開盤價'][i+1]))/float(df['開盤價'][i+1])

            if U > upper :
                list_.append(1)
            else :
                list_.append(0)
                
        except Exception as e:
            print(e , i)
            list_.append(0)
        i+=1
    df[label] = list_
def shift_drop_estimate(df, day , upper ):
    i=0
    label = "pre" + str(day) + "d" + str(upper)
    list_ = [0 for x in range(day)]
    while i < (len(df) -day):
        try :
        
            U =(min(df['最低價'][i+1:i+day+1])-float(df['開盤價'][i+1]))/float(df['開盤價'][i+1])

            if -U > upper :
                list_.append(1)
            else :
                list_.append(0)
                
        except Exception as e:
            print(e , i)
            list_.append(0)
        i+=1
    df[label] = list_

def 合併預測資料(xt,yt,xte,yte,ytp,ytep,t):
   
    xt["label"]=yt
    xt["預測結果"]=ytp
    
    
    xte["label"]=yte
    xte["預測結果"]=ytep
    
    ALL=xt.append(xte)

    ALL["昨天預測結果"]=ALL["預測結果"].shift(1)
    ALL["前天預測結果"]=ALL["預測結果"].shift(2)
    ALL["大前天預測結果"]=ALL["預測結果"].shift(3)
    #ALL["昨天是否預測正確"]=(ALL["label"].shift(1)==ALL["昨天預測結果"]).astype(int)
    #ALL["前天是否預測正確"]=ALL["昨天是否預測正確"].shift(1)
    #ALL["大前天是否預測正確"]=ALL["昨天是否預測正確"].shift(2)
    ALL=ALL.dropna(subset=["昨天預測結果", "前天預測結果","大前天預測結果"])
    ALL["date"]=t["date"]
    ALL["證券代號"]=t["證券代號"]
    return ALL
TWSE_URL = 'http://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
TPEX_URL = 'http://isin.twse.com.tw/isin/C_public.jsp?strMode=4'

columns = ['dtype', 'code', 'name', '國際證券辨識號碼', '上市日', '市場別', '產業別', 'CFI']

items = []
for url in [TWSE_URL, TPEX_URL]:
    response_dom = pq(url)
    for tr in response_dom('.h4 tr:eq(0)').next_all().items():
        if tr('b'):
            dtype = tr.text()
        else:
            row = [td.text() for td in tr('td').items()]
            code, name = row[0].split('\u3000')
            items.append(dict(zip(columns, [dtype, code, name, *row[1: -1]])))

data = pd.DataFrame(items)

HTML(data.head().to_html())

def monthly_report(year, month):
    
    # 假如是西元，轉成民國
    if year > 1990:
        year -= 1911
    
    url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'_0.html'
    if year <= 98:
        url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'.html'
    
    # 偽瀏覽器
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
    # 下載該年月的網站，並用pandas轉換成 dataframe
    r = requests.get(url, headers=headers)
    r.encoding = 'big5'

    dfs = pd.read_html(StringIO(r.text), encoding='big-5')

    df = pd.concat([df for df in dfs if df.shape[1] <= 11 and df.shape[1] > 5])
    
    if 'levels' in dir(df.columns):
        df.columns = df.columns.get_level_values(1)
    else:
        df = df[list(range(0,10))]
        column_index = df.index[(df[0] == '公司代號')][0]
        df.columns = df.iloc[column_index]
    
    df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')
    df = df[~df['當月營收'].isnull()]
    df = df[df['公司代號'] != '合計']
    
    # 偽停頓
    time.sleep(5)

    return df

def 加入產概(model,產概):
    model=model.merge(產概, how='left', left_on='證券代號', right_on='代號').drop(columns=["代號"])
    return model
def XGBBEST(X_train,y_train):
    xgbc = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.8,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.01, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=400, n_jobs=4, nthread=4, num_parallel_tree=1,
              random_state=27, reg_alpha=0, reg_lambda=1,
              seed=27, subsample=0.8, tree_method='exact',
              validate_parameters=1, verbosity=None).fit(X_train,y_train)
    return xgbc
def draw_precision(testp,predicted,y_test):
    testp["label"]=predicted
    testp["actual"]=y_test
    DP=pd.DataFrame({"日期":[],"預測數量":[],"precision":[]})
    for i in testp["date"].value_counts().index:
        temp=testp[testp["date"]==i]
        DP=DP.append(pd.DataFrame({"日期":[i],"預測數量":[temp["label"].sum()],"precision":[temp[temp["label"]==1]["actual"].sum()/temp["label"].sum()]}))
    DP=DP.sort_values(by=["日期"])
    BS=pd.read_csv("Market_index.csv")
    BS=BS[BS["Index"]=="發行量加權股價指數"]
    BS["date"]=pd.to_datetime(BS["date"],format = '%Y%m%d')
    DP["日期"]=pd.to_datetime(DP["日期"],format = '%Y%m%d')
    DP=DP.merge(BS, how='left', left_on='日期', right_on='date')
    fig, ax = plt.subplots(figsize=(15,4))

    ax.plot(DP["日期"],DP["precision"], marker='o')
    ax.set_xlabel('date')
    ax.set_ylabel('precision')
    ax2 = ax.twinx()
    ax2.plot(DP["日期"],DP["收盤"], color='red')
    plt.title('precision chart')
    ax2.set_ylabel('stock index')
    plt.grid(True)
    DP.to_excel("precision表.xlsx",index=None)
    testp.to_excel("testp.xlsx",index=None)
    print("precision表.xlsx 表格輸出成功")
    plt.show()
def 業績試算(testp,Stock,停利,停損):
    
    業績=pd.merge(testp, Stock, on=['證券代號', 'date'])[["開盤價","收盤價","date","label","證券代號"]].sort_values("date")
    money=0
    績效表=pd.DataFrame({"證券代號":[],"進場日期":[],"出場日期":[],"進場價格":[],"出場價格":[],"目前賺":[],"投資成本":[],"出場回收":[],"賺":[]})
    for i in 業績["證券代號"].value_counts().index:
        temp=業績[業績["證券代號"]==i].sort_values("date").reset_index(drop=True)
        k=0
        #print(i)

        while(k+1<len(temp["證券代號"])):

            if (temp["label"][k]==1):
                currentP=temp["開盤價"][k+1]
                current_date=temp["date"][k+1]
                share=math.floor(50/currentP)
                x=k
                while(x<len(temp["證券代號"])):
                    if(temp["收盤價"][x]/currentP>停利):                   
                        money+=currentP*share*(停利-1.006)*1000
                        績效表=績效表.append(pd.DataFrame({"證券代號":[i],"進場日期":[str(current_date)],
                                                     "出場日期":[str(temp["date"][x])],"進場價格":[str(currentP)],
                                                     "出場價格":[str(temp["收盤價"][x])],"目前賺":[str(money)],
                                                     "投資成本":[currentP*share*1000],
                                                     "出場回收":[temp["收盤價"][x]*share*1000],
                                                     "賺":[currentP*share*(停利-1.006)*1000]}))

                        break
                    elif(temp["收盤價"][x]/currentP<停損):
                        money-=currentP*share*(1.006-停損)*1000
                        績效表=績效表.append(pd.DataFrame({"證券代號":[i],"進場日期":[str(current_date)],
                                                     "出場日期":[str(temp["date"][x])],"進場價格":[str(currentP)],
                                                     "出場價格":[str(temp["收盤價"][x])],"目前賺":[str(money)],
                                                     "投資成本":[currentP*share*1000],
                                                     "出場回收":[temp["收盤價"][x]*share*1000],
                                                     "賺":[currentP*share*(停損-1.006)*1000]}))

                        break
                    if(x==len(temp["證券代號"])-1):
                        money+=(temp["收盤價"][x]-currentP)*share*1000  
                        績效表=績效表.append(pd.DataFrame({"證券代號":[i],"進場日期":[str(current_date)],"出場日期":[str(temp["date"][x])],"進場價格":[str(currentP)],"出場價格":[str(temp["收盤價"][x])],
                                                     "目前賺":[str(money)],"投資成本":[currentP*share*1000],
                                                     "出場回收":[temp["收盤價"][x]*share*1000],
                                                     "賺":[temp["收盤價"][x]*share*1000-currentP*share*1000*1.006]}))

                        break
                    x+=1
            k+=1
    return 績效表 

def 當沖業績試算(testp,Stock,只買一檔):
    if(只買一檔==0):
        業績=pd.merge(testp, Stock, on=['證券代號', 'date'])[["開盤價","收盤價","date","label","證券代號"]].sort_values("date")
        money=0
        績效表=pd.DataFrame({"證券代號":[],"進場日期":[],"出場日期":[],"進場價格":[],"出場價格":[],"目前賺":[],"投資成本":[],"出場回收":[],"賺":[]})
        for i in 業績["證券代號"].value_counts().index:
            temp=業績[業績["證券代號"]==i].sort_values("date").reset_index(drop=True)
            k=0
            #print(i)

            while(k+1<len(temp["證券代號"])):

                if (temp["label"][k]==1):
                    currentP=temp["開盤價"][k+1]
                    current_date=temp["date"][k+1]
                    share=math.floor(100/currentP)


                    money+=(temp["收盤價"][k+1]-currentP*1.006)*share*1000
                    績效表=績效表.append(pd.DataFrame({"證券代號":[i],"進場日期":[str(current_date)],
                                                         "出場日期":[current_date],"進場價格":[str(currentP)],
                                                         "出場價格":[temp["收盤價"][k+1]],"目前賺":[str(money)],
                                                         "投資成本":[currentP*share*1000],
                                                         "出場回收":[temp["收盤價"][k+1]*share*1000],
                                                         "賺":[(temp["收盤價"][k+1]-currentP*1.006)*share*1000]}))

                k+=1
        return 績效表
    
    else:
        
        業績=pd.merge(testp, Stock, on=['證券代號', 'date'])[["開盤價","收盤價","date","label","證券代號"]].sort_values("date")
        money=0
        績效表=pd.DataFrame({"證券代號":[],"進場日期":[],"出場日期":[],"進場價格":[],"出場價格":[],"目前賺":[],"投資成本":[],"出場回收":[],"賺":[]})
        for d in 業績["date"].value_counts().index[1:].sort_values():
            try:
                temp=業績[業績["date"]==d]
                stockname=((temp.sort_values("label",ascending=False)).reset_index(drop=True))["證券代號"][0]
                temp=業績[業績["證券代號"]==stockname].sort_values("date").reset_index(drop=True)
                k=(temp[temp["date"]==d].index)[0]
                currentP=temp["開盤價"][k+1]
                current_date=temp["date"][k+1]
                share=math.floor(1000/currentP)


                money+=(temp["收盤價"][k+1]-currentP*1.006)*share*1000
                績效表=績效表.append(pd.DataFrame({"證券代號":[stockname],"進場日期":[str(current_date)],
                                                                 "出場日期":[current_date],"進場價格":[str(currentP)],
                                                                 "出場價格":[temp["收盤價"][k+1]],"目前賺":[str(money)],
                                                                 "投資成本":[currentP*share*1000],
                                                                 "出場回收":[temp["收盤價"][k+1]*share*1000],
                                                                 "賺":[(temp["收盤價"][k+1]-currentP*1.006)*share*1000]}))
            except:
                continue
        return 績效表
        

def 當沖放空業績試算(testp,Stock):
    
    業績=pd.merge(testp, Stock, on=['證券代號', 'date'])[["開盤價","收盤價","date","label","證券代號"]].sort_values("date")
    money=0
    績效表=pd.DataFrame({"證券代號":[],"進場日期":[],"出場日期":[],"進場價格":[],"出場價格":[],"目前賺":[],"投資成本":[],"出場回收":[],"賺":[]})
    for i in 業績["證券代號"].value_counts().index:
        temp=業績[業績["證券代號"]==i].sort_values("date").reset_index(drop=True)
        k=0
        #print(i)

        while(k+1<len(temp["證券代號"])):

            if (temp["label"][k]==1):
                currentP=temp["開盤價"][k+1]
                current_date=temp["date"][k+1]
                share=math.floor(1000/currentP)

                                       
                money+=(currentP-temp["收盤價"][k+1]*1.006)*share*1000
                績效表=績效表.append(pd.DataFrame({"證券代號":[i],"進場日期":[str(current_date)],
                                                     "出場日期":[current_date],"進場價格":[str(currentP)],
                                                     "出場價格":[temp["收盤價"][k+1]],"目前賺":[str(money)],
                                                     "投資成本":[currentP*share*1000],
                                                     "出場回收":[temp["收盤價"][k+1]*share*1000],
                                                     "賺":[(currentP-temp["收盤價"][k+1]*1.006)*share*1000]}))

            k+=1
    return 績效表 

def 每日業績(績效表):
    A=pd.DataFrame({"進場日期":績效表.groupby(by=["進場日期"])["投資成本"].sum().index,"投資成本":績效表.groupby(by=["進場日期"])["投資成本"].sum().values})
    B=pd.DataFrame({"出場日期":績效表.groupby(by=["出場日期"])["出場回收"].sum().index,"出場回收":績效表.groupby(by=["出場日期"])["出場回收"].sum().values})
    date_業績=A.merge(B,how="outer", left_on='進場日期', right_on='出場日期').fillna(0)
    date_業績["資本額"]=""
    last=0
    i=0
    while(i<len(date_業績["進場日期"])):
        date_業績["資本額"][i]=last-date_業績["投資成本"][i]+date_業績["出場回收"][i]
        last=date_業績["資本額"][i]
        i+=1
    date_業績.to_excel("date_業績.xlsx")
    return date_業績



def 資金流動_本益比_龍頭(Stock):
    股數=pd.read_excel("股數.xlsx")
    Stock=Stock.merge(股數,how="left",left_on="證券代號",right_on="股票代號")
    Stock["市值"]=Stock["收盤價"]*Stock["股數"].astype(float)
    市值增加=pd.DataFrame({"證券代號":[],"date":[],"市值增加":[]})
    for i in Stock["證券代號"].value_counts().index:
        temp=Stock[Stock["證券代號"]==i].sort_values(["date"]).reset_index(drop=True)
        temp["市值增加"]=temp["市值"].shift(1)-temp["市值"]
        temp=temp[["證券代號","date","市值增加"]]
        市值增加=市值增加.append(temp)
    Stock=Stock.merge(市值增加,how="left",left_on=["證券代號","date"],right_on=["證券代號","date"])
    產業別=pd.read_excel("產業別與營收.xlsx")
    產業別=產業別[["公司代號","產業別"]]
    產業別=產業別.drop_duplicates()
    Stock["證券代號"]=Stock["證券代號"].astype(int).astype(str)
    Stock_產業=Stock.merge(產業別,how="left",left_on="證券代號",right_on="公司代號").drop(columns=["股數","公司代號","股票代號"])
    產業資金=pd.DataFrame({"產業別":[],"date":[],"產業資金增加":[],"市場資金增加":[]})
    for i in Stock_產業["date"].value_counts().index:
        temp=Stock_產業[Stock_產業["date"]==i]
        產業資金=產業資金.append(pd.DataFrame({"產業別":temp.groupby("產業別")["市值增加"].sum().index,"date":np.full(len(temp.groupby("產業別")["市值增加"].sum().index), i),"產業資金增加":temp.groupby("產業別")["市值增加"].sum().values,"市場資金增加":np.full(len(temp.groupby("產業別")["市值增加"].sum().index), temp.groupby("產業別")["市值增加"].sum().sum())}))
    Stock_產業=Stock_產業.merge(產業資金,how="left",left_on=["產業別","date"],right_on=["產業別","date"])
    Stock_產業["個股資金增加占產業比之絕對值"]=abs(Stock_產業["市值增加"]/Stock_產業["產業資金增加"])
    Stock_產業["產業資金增價占市場比之絕對值"]=abs(Stock_產業["產業資金增加"]/Stock_產業["市場資金增加"])
    
    本益比=pd.DataFrame({"產業別":[],"date":[],"產業平均本益比":[]})
    A=Stock_產業.groupby(["產業別","date"])["本益比"].mean().index.get_level_values('產業別')
    B=Stock_產業.groupby(["產業別","date"])["本益比"].mean().index.get_level_values('date')
    C=Stock_產業.groupby(["產業別","date"])["本益比"].mean().values
    本益比=本益比.append(pd.DataFrame({"產業別":A,"date":B,"產業平均本益比":C}))
    Stock_產業=Stock_產業.merge(本益比,how="left",left_on=["產業別","date"],right_on=["產業別","date"])
    Stock_產業["個股對產業本益比比率"]=Stock_產業["本益比"]/Stock_產業["產業平均本益比"]
    
    
    產業排名=pd.DataFrame({"證券代號":[],"date":[],"個股產業排名":[]})
    for 產業 in Stock_產業["產業別"].value_counts().index:
        temp=Stock_產業[Stock_產業["產業別"]==產業][["證券代號","產業別","市值","date"]]
        for 日期 in temp["date"].value_counts().index:
            temp2=temp[temp["date"]==日期]
            temp2=temp2.sort_values(["市值"],ascending=False).reset_index(drop=True)
            產業排名=產業排名.append(pd.DataFrame({"證券代號":temp2["證券代號"],"date":temp2["date"],"個股產業排名":temp2.index}))
    Stock_產業=Stock_產業.merge(產業排名,how="left",left_on=["證券代號","date"],right_on=["證券代號","date"])
    
    Stock_產業["龍頭"]=""
    Stock_產業.龍頭[Stock_產業["個股產業排名"]==0]=1
    Stock_產業.龍頭[Stock_產業["個股產業排名"]!=0]=0
    霸主=pd.DataFrame({"證券代號":[],"date":[],"成為霸主":[]})
    temp0=Stock_產業[["證券代號","date","龍頭"]]
    for i in Stock_產業["證券代號"].value_counts().index:
        temp=temp0[temp0["證券代號"]==i].sort_values(["date"])
        temp["成為霸主"]=""
        temp["成為霸主"]=temp["龍頭"]-temp["龍頭"].shift(1)
        霸主=霸主.append(pd.DataFrame({"證券代號":temp["證券代號"],"date":temp["date"],"成為霸主":temp["成為霸主"]}))
    Stock_產業=Stock_產業.merge(霸主,how="left",left_on=["證券代號","date"],right_on=["證券代號","date"])
    
    return Stock_產業[['證券代號','date',
       '市值', '市值增加', '產業別', '產業資金增加', '市場資金增加', '個股資金增加占產業比之絕對值',
       '產業資金增價占市場比之絕對值', '產業平均本益比', '個股對產業本益比比率', '個股產業排名', '龍頭', '成為霸主']]
def Detrend_AddFeature(data):
    data=data.sort_values("Date")
    data=data[data["Date"]>=20010101]
    data=data[["Date","Close","High","Low","Open","STOCK_ID"]]
    股票代號=data["STOCK_ID"].value_counts().index
    temp=pd.DataFrame({"Date":[],"Close":[],"High":[],"Low":[],"Open":[],"STOCK_ID":[],"漲幅":[],"y1漲幅":[],
                       "y2漲幅":[], "y3漲幅":[], "y4漲幅":[],"5日內最大漲幅":[]})
    for i in 股票代號:
        temp2=data[data.STOCK_ID==i].sort_values("Date")
        temp2["漲幅"]=temp2["Close"]/temp2["Close"].shift(1)
        temp2["y1漲幅"]=temp2["Close"].shift(1)/temp2["Close"].shift(2)
        temp2["y2漲幅"]=temp2["Close"].shift(2)/temp2["Close"].shift(3)
        temp2["y3漲幅"]=temp2["Close"].shift(3)/temp2["Close"].shift(4)
        temp2["y4漲幅"]=temp2["Close"].shift(4)/temp2["Close"].shift(5)
        temp2=temp2.reset_index(drop=True)

        temp=temp.append(temp2)


    from tqdm import tqdm 
    import time

    temp3=temp[0:0]
    temp2["5日內最大漲幅"]="" 
    temp2["5日內最大跌幅"]=""
    t=0
    with tqdm(total=len(股票代號)) as pbar:

        for i in 股票代號:
            temp2=temp[temp.STOCK_ID==i].sort_values("Date")
            temp4=pd.DataFrame({"1":temp2["Close"].shift(-1),"2":temp2["Close"].shift(-2),"3":temp2["Close"].shift(-3),
                        "4":temp2["Close"].shift(-4),"5":temp2["Close"].shift(-5)})
            temp5=pd.DataFrame({"1":temp2["Low"].shift(-1),"2":temp2["Low"].shift(-2),"3":temp2["Low"].shift(-3),
                        "4":temp2["Low"].shift(-4),"5":temp2["Low"].shift(-5)})
            MAX=temp4.max(axis=1)
            MIN=temp5.min(axis=1)
            temp2["5日內最大漲幅"]=MAX
            temp2["5日內最大跌幅"]=MIN
            
            g=4
            temp6=temp2
            temp6['lognorm'] = np.log(temp2['漲幅'])
            temp6["Entropy+"]=""
            temp6["Entropy-"]=""

            while g<len(temp2["漲幅"]):
                temp7=temp6["lognorm"][g-4:g+1]
                temp6["Entropy+"][g]=entropy(temp7[temp7>0], base=2)
                temp6["Entropy-"][g]=entropy(temp7[temp7<0], base=2)
                g+=1
            temp6=temp6[4:]
            temp3=temp3.append(temp6[:-5])
            #print(temp3)
            pbar.update(1)
    temp3["5日內最大漲幅"]=temp3["5日內最大漲幅"]/temp3["Close"]
    temp3["5日內最大跌幅"]=temp3["5日內最大跌幅"]/temp3["Close"]
    temp3["Open_PreClose"]=temp3["Open"]/temp3["Close"].shift(1)
    temp3["High_Open"]=temp3["High"]/temp3["Open"]
    temp3["Low_Open"]=temp3["Low"]/temp3["Open"]
    temp3["Close_Open"]=temp3["Close"]/temp3["Open"]
    temp3=temp3.dropna()
    temp3=temp3[temp3["5日內最大漲幅"]<1.62]
    temp3["Label"]=(temp3["5日內最大漲幅"]>1.036).astype(int)-(temp3["5日內最大跌幅"]<0.97).astype(int)
    
   
    temp3["5日內大於一點八趴"]=(temp3["5日內最大漲幅"]>1.024).astype(int)
    print(temp3["5日內大於一點八趴"].value_counts())
    for i in temp3.columns: 
        temp3=temp3[~temp3[i].isin([np.inf, -np.inf])]

    return temp3
